## Задание
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = \['python', 'парсинг'\]

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [1]:
import requests
import pandas
from bs4 import BeautifulSoup

KEYWORDS = ['python', 'парсинг']
url = 'https://habr.com/ru/all/'

In [2]:
def checkTag(tags):
    for keyword in KEYWORDS:
        if(keyword.lower() not in tags and keyword != KEYWORDS[-1]):
            return False
    return True

In [3]:
import time
def getAllInfo(url, query, pages):
    habrPosts = pandas.DataFrame()
    
    for i in range(pages):
        page = 'page' + str(i + 1)
        hubrPage = requests.get(url+page)
        if (hubrPage.status_code != 200):
            print('Something Wrong in Web Site')
            break
        #time.sleep(0.3) #Ругается UnboundLocalError: local variable 'time' referenced before assignment
        soup = BeautifulSoup(hubrPage.text, 'html.parser')
        blogs = soup.find_all('article', class_='post post_preview')
        for blog in blogs:
            tags = []
            for listTags in blog.find('ul').find_all('a'):
                tags.append(listTags.text.lower())

            if(not checkTag(tags)):
                continue
            try:
                title = blog.find('a', class_='post__title_link').text
                link = blog.find('a', class_='post__title_link').get('href')
                text = blog.find('div', class_='post__text post__text-html post__text_v1').text.strip()
                time = blog.find('span', class_='post__time').text
            except AttributeError:
                print('Corrupted data on ',page)
                print("Can't read ",link) 
                break
                
            info = [time, title, text, link]
            row = {'time': info[0], 'title': info[1], 'text': info[2], 'link': info[3]}
            habrPosts = pandas.concat([habrPosts, pandas.DataFrame([row])]) 
                 
    return habrPosts

all_links = getAllInfo(url, KEYWORDS, 5)
all_links

,time,title,text,link
0,вчера в 18:11,Немного про трекинг и сервис переходов Admitad,"Привет, меня зовут Александр, я тимлид команды...",https://habr.com/ru/company/admitad_gmbh/blog/...
